# How to find and group outliers in a logstream, based on behaviour

## Context

Your boss asked you to find suspect users among the few thousands of customers using your web site every day.

Where do you start?! Easy! AI, Tensorflow, machine learning, and other buzzwords like so will solve the issue! When all logs are categorized, it's quite easy right?! You know what is going on in your logs right?

... most probably not, especially if your site is generating few gigabytes of logs per day, there is just too much logs to know everything that is going one.

Before even starting to search for suspicious users, step 1 would be starting by identifying what it means to be suspicious.

    Suspicious normally means that it stands out from the normal.
    
Ok, step 2: what is normal?

    Normal behaviour consist of expected usage. Anything outside of this can be considered abnormal.

... We are not much more advanced because now we need to go over each action, and identify what is normal, and what is not, and assign a score to each.

The second issue with this is that normal behaviour will be definition be way more likely than the abnormal one, which will skew the results into making any machine learning model think that 99.9% of the usage is normal, which a margin of error of + or - 5%, flooding your analysis with noise.

The classical approach is to use a simple rule based system. By setting few triggers, we can find out account attackers. For small scale or simple systems, this will is likely to be enough, but not for large scale and complex systems.

One other hypothesis would be to use Hidden Markov Chains, but for HNN, we need to know what we are searching for.
Logs are also noisy: users will not simply go from A to B then C. they might crawl the whole site before reaching C, which complicates the creation of HMM.

Another hypothesis would be to filter out known noise, like when we want to do sentiment analysis, English stop words are just too common and does not tell us anything useful... but what if these stop words are useless in small quantities, but an indicator of an issue in large quantities? ... and how much is too much?


## Information Theory To The Rescue

Information Theory is normally used for signal analysis and compression. By using Surprisal Analysis and Kullback–Leibler divergence, we can group similar behaviour togethers.

* [Information Content](https://en.wikipedia.org/wiki/Information_content)
* [Surprisal Analysis](https://en.wikipedia.org/wiki/Surprisal_analysis)
* [Kullback–Leibler divergence](https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence)
* [Example of Surprisal](http://www.umsl.edu/~fraundorfp/egsurpri.html)

Basically, Surprisal Analysis is a way to measure (in bits) how surprised we will be by looking at something. Surprisal are calculated using Log base 2, which makes it easy to calculate because log are additive when probabilities are multiplied.

For example:
* There is 1/6 chance to roll a 6 die, gives us 2.58 bits of information (-log2(1/6) ~= 2.4849 bits). If you are playing D&D, you will be happily surprised.
* Not rolling a 6 on a die represents 0.263 bits (-log2(5/6) ~= 0.263 bits). So rolling anything but a 6 in that same D&D game will most likely leave you dissapointed. 
* The chance of rolling 10 x 6 in a row on a die is 1/60466176 (10 * -log2(1/6) ~= 25.85 bits). This is highly unlikely.

By assigning a surprisal value to each action taken over all actions taken by all users gives us an idea how much surprisal by seeing any of them happening, and by adding the score of all actions of a user, we can get an idea how "normal" was all his/her actions.

This is what this notebook intend to demonstrate: Using Surprisal analysis, we will assign a score to actions, and by adding up the score of each action, identify series of actions that are unlikely to occur.

# But first, we need some logs

What is explained in this notebook can be applied to real logs, but for the experimentation, I added a "blackbox" library that will help to generate our users and the logs of these users based on a determined probability distribution.

## Caviat: I had to cheat to achieve the desired results

Because I am generating the logs through a known distribution, I am also able to know the type of user of each user, which makes it easy later on to debug and know how wrong I am.

This cheat was also necessary to get a probability distribution of the classification function of this notebook. This probability distribution is likely to be different for live environment, and should be tweeked.

| | Positive | Negative |
| -- | -- | -- |
| True | 0.875 | 0.102 |
| False | 0.125 | 0.898 |

I am using this probability distribution, along with the Bayes Theorem to update my belief that each tested candidates truely belong in the tested behaviour classification.

## Different User Profiles Generated By The Library

The following profiles are generated by the library.

Normal users
* Buyer
* Merchants

Abnormal users:
* Scraper bots
* Spammers
* fraudster
* Account Attackers

Buyers and merchants represent 98% of our logs. Leaving 2% to the abnormal users. However, the actions taken by each users being on a probability distribution, it is possible to see an "attakcer" user being classified as a user, because that attacked might have had a change of heart and didn't attack after all.

In [1]:
## This cell initialize the libraries needed for this notebook, 
## along with the blackbox functions used to generate the users and logs

import random
import numpy as np
import pandas as pd
from datetime import datetime, date, time, timedelta
from math import log, pow

%load_ext autoreload
%autoreload 2

from blackbox import distribution
from blackbox import generate_userlist, generate_logs, cheat_calculate_hit_rate, cheat_lookup_all_users

magic = distribution()

# Generating The User Database

The following cell generates our user database.

You will notice in this notebook that we are reinitializing the random seed quite often, just to keep consistence during the testing. Set the *random_seed* to False to randomize everything.

In [2]:
random_seed = 42

if random_seed:
    random.seed(random_seed)

## We define how many users here
number_of_new_users = 5000


existing_users = [] # Later on, we can add users to our list by supplying it to the generate_userlist function
user_lists = generate_userlist(number_of_new_users, existing_users)

print(len(user_lists), user_lists[:5])

5000 ['merchant', 'buyer', 'buyer', 'buyer', 'merchant']


# Generating Logs For Day 1

Note: The more users we have, the more log events will be generated. The probability distribution of each user ensures that they will start with a defined action, crawl the site following a defined pattern, and logout eventually, until the end of the day.

In [3]:
%%time
if random_seed:
    random.seed(random_seed)

start_time = datetime(2019,1,1,0,0)
day1_logs = generate_logs(user_lists, start_time)

print(len(day1_logs), 'logs event generated for', len(user_lists), 'users')

126470 logs event generated for 5000 users
CPU times: user 48 s, sys: 201 ms, total: 48.2 s
Wall time: 48.1 s


## Transforming the logs in a pandas dataframe

The transition surprisal lookup table used in this notebook calculates scores based on the movements of the users between each actions. For example:

* login (success) -> view_items (success) will result in a low surpisal value
* login (fail) -> buy_item (success) never happened. If this sequence happen, this should be a huge red flag.

In [4]:
def transform_logs_to_pandas(logs):
    data = pd.DataFrame(np.array(logs), columns=['time', 'user', 'path', 'status', 'uidx', 'realtype'])
    
    data['prev_path'] = data.groupby(['user'])['path'].shift(1)
    data['prev_path'] = data['prev_path'].fillna("")

    data['prev_status'] = data.groupby(['user'])['status'].shift(1)
    data['prev_status'] = data['prev_status'].fillna("")
    return data
    
day1_data = transform_logs_to_pandas(day1_logs)

print(day1_data.loc[(day1_data['path'] == 'login') & (day1_data['status'] == 'fail')].head(10))

                     time           user   path status  uidx   realtype  \
372   2019-01-01 00:13:20   merchant1478  login   fail  1478   merchant   
1562  2019-01-01 00:47:33      buyer2547  login   fail  2547      buyer   
1808  2019-01-01 00:57:21   merchant1468  login   fail  1468   merchant   
2184  2019-01-01 01:07:53   merchant3721  login   fail  3721   merchant   
2543  2019-01-01 01:15:41      buyer3774  login   fail  3774      buyer   
2642  2019-01-01 01:19:23   merchant2417  login   fail  2417   merchant   
3001  2019-01-01 01:27:53    merchant955  login   fail   955   merchant   
3310  2019-01-01 01:35:47      buyer4895  login   fail  4895      buyer   
3329  2019-01-01 01:36:08      buyer4895  login   fail  4895      buyer   
3530  2019-01-01 01:39:52  fraudster4405  login   fail  4405  fraudster   

     prev_path prev_status  
372                         
1562                        
1808                        
2184                        
2543                        


The following cell generates the transition surprisal lookup table used to score each actions taken by the users.

The format is as follow:

```
['current path'],['previous path']: {
    'fail': 0, # How many time this action transition failed. (ex. View Items: Success, from: Login: Success)
    'success': 13, # How many time this action transition succeded (ex. View Items: Fail, from: Login: Success)
    'fsurprisal': 11.266786540694902, # Surprisal value if there is a failure happens
    'ssurprisal': 7.56634682255381 # Surprisal value if that action is successful.
    }
```

The surprisal value is directly related to the likelihood of an actions happening. If an actions is observed successfully few million times, then the successful surprisal value will be really low. However, the failure surprisal will be much higher if it never happens.

In [5]:
def init_transition_surprisal_lookup(data, key, prev_key, feature, success):
    surprisal = {}

    for pkey in data[key].unique():
        data_for_pkey = data.loc[(data[key] == pkey)]
        denum = len(data.loc[(data[key] == pkey)])

        for ppkey in data_for_pkey[prev_key].unique():
            ds = data_for_pkey.loc[(data_for_pkey[prev_key] == ppkey) & (data_for_pkey[feature] == success)]
            df = data_for_pkey.loc[(data_for_pkey[prev_key] == ppkey) & (data_for_pkey[feature] != success)]

            dsuccess = len(ds) * 1.0
            dfail = len(df) * 1.0

            if dsuccess == 0:
                dsuccess = 1.0 

            if dfail == 0:
                dfail = 1.0

            if (pkey not in surprisal.keys()):
                surprisal[pkey] = {}

            surprisal[pkey][ppkey] = {
                'success': len(ds), 
                'fail': len(df), 
                'ssurprisal': log(1/(dsuccess / denum),2), 
                'fsurprisal': log(1/(dfail / denum),2),
            }
    return surprisal

transition_surprisal = init_transition_surprisal_lookup(day1_data, 'path', 'prev_path', 'status', 'success')

In [6]:
def get_transition_surprisal(path, prev_path, surprisal, data):
    if path not in list(surprisal.keys()):
        denum = len(data)
        return {
            'fail': 0,
            'success': 0,
            'ssurprisal': log(1/(1/denum),2),
            'fsurprisal': log(1/(1/denum),2),
        }
    else:
        if prev_path not in list(surprisal[path].keys()):
            denum = len(data.loc[(data['path'] == path)])
            return {
                'fail': 0,
                'success': 0,
                'ssurprisal': log(1/(1/denum),2),
                'fsurprisal': log(1/(1/denum),2),
            }
        else:
            return surprisal[path][prev_path]

get_transition_surprisal('buy_item', 'login', transition_surprisal, day1_data)

{'fail': 3,
 'fsurprisal': 11.046669305196481,
 'ssurprisal': 7.0770429542399995,
 'success': 47}

In [7]:
def get_user_transition_score(data, surprisal, key, feature, success_val):
    accumulator = {}
    key_last_path = {}
    
    for index,row in data.iterrows():
        if row[key] not in key_last_path.keys():
            key_last_path[row[key]] = ""
            
        if row[key] not in accumulator.keys():
            accumulator[row[key]] = {k:0 for k in data[feature].unique()}
            
        if row[feature] is success_val:
            accumulator[row[key]][row[feature]] += get_transition_surprisal(row[feature],key_last_path[row[key]], surprisal, data)['ssurprisal']
        else:
            accumulator[row[key]][row[feature]] += get_transition_surprisal(row[feature],key_last_path[row[key]], surprisal, data)['fsurprisal']

        key_last_path[row[key]] = row[feature]
                                    
    return accumulator


user_transition_score = get_user_transition_score(day1_data, transition_surprisal, 'user', 'path', 'success')

print(user_transition_score['fraudster96'])
print(user_transition_score['buyer402'])

{'login': 5.109480743474441, 'view_item': 0, 'buy_item': 12.631631805917637, 'sell_item': 0, 'logout': 11.615169813843604, 'end': 12.281930026955443, 'comment': 0, 'home': 0, 'view_profile': 7.219168520462162, 'bank_modify': 0, 'update_address': 0, 'update_email': 0, 'password_reset': 0, 'payment_modify': 0}
{'login': 5.109480743474441, 'view_item': 49.06048316083863, 'buy_item': 16.701021187614202, 'sell_item': 0, 'logout': 11.615169813843604, 'end': 12.281930026955443, 'comment': 0, 'home': 0, 'view_profile': 0, 'bank_modify': 0, 'update_address': 0, 'update_email': 0, 'password_reset': 0, 'payment_modify': 0}


If we go through the logs of the day, can we identify outliers?

To do so, the idea is to look at each action being taken by each user, and add the relevant value from the surprisal lookup table.

That said, I did read a lot about information theory and surprisal analysis, and this most probably not how it is supposed to be used, and the calculation is most probably wrong... but this mistake is quite useful

In [8]:
cumulative_score = [[v,sum(user_transition_score[v].values())] for v in [k for k in list(user_transition_score.keys())]]

df_cumulative_score = pd.DataFrame(cumulative_score, columns=['user', 'surprisal'])

avg = df_cumulative_score['surprisal'].mean()
std = df_cumulative_score['surprisal'].std()
df_cumulative_score['z'] = (df_cumulative_score['surprisal'] - avg) / std


In [9]:
df_cumulative_score.loc[df_cumulative_score['z'] >= 2].sort_values(by=['surprisal'], ascending=False)

,user,surprisal,z
1426,bot2306,52585.165443,33.753712
3391,bot900,49962.954676,32.064342
3458,bot4691,48004.151536,30.802376
2651,bot2816,37804.612110,24.231284
3903,bot4577,30306.140606,19.400366
2739,bot2493,22324.098653,14.257905
1427,bot776,20924.178403,13.356001
3547,bot672,18764.611507,11.964692
1705,bot2886,17527.472681,11.167661
568,bot1946,15688.042586,9.982601


In [10]:
df_cumulative_score.sort_values(by=['surprisal'], ascending=False).tail()

,user,surprisal,z
4361,attacker4573,12.28193,-0.116567
4374,merchant2290,12.28193,-0.116567
4439,fraudster4103,12.28193,-0.116567
428,merchant3785,12.28193,-0.116567
1711,merchant63,12.28193,-0.116567


In [11]:
np.seterr(divide='ignore', invalid='ignore', over='ignore')

if random_seed:
    np.random.seed(random_seed)

flat_status, flat_lookup = cheat_calculate_hit_rate(day1_data, user_transition_score)

print('count', True, flat_status[True])
print('count', False, flat_status[False])

print('percent', True, flat_lookup[True])
print('percent', False, flat_lookup[False])

count True {True: 163, False: 26}
count False {True: 437, False: 2974}
percent True {True: 0.8624338624338624, False: 0.12811492231017296}
percent False {True: 0.13756613756613756, False: 0.871885077689827}


Can we cluster users together?


We will use two lists: 
- unclassified_users, which is a copy of the original user_score list, but not classified yet
- behaviour_types: this is a list of hashes, the key being the user type (normally type + n), and the value of that hash is an array with all the profiles of the users classified under that type

With these 2 lists, here is the idea:
- From the unclassified_users list, we will pick one at random
- Then we will pick 100 users at random, and try to find 10 that matches the behaviour. It's ok if we don't find any
- I make the hypothesis that we have 10 kind of users, so my prior probability is 0.1
- Then we check in all the keys under behaviour_types and compare our found random candidates with max 10 random candidates from that behaviour type. For the experimental data, We know that the True Positive probability is 51.5%, and the True Negative rate is 11.2%. By doing enough tests, we should be able to update our belief that we are comparing matching or different profiles

In [12]:
def update_probability(prior_probability, distribution, test_result):
    
    # What is our success rate for this test_result?
    likelihood_of_being_right = distribution[test_result][True]
    likelihood_of_being_wrong = distribution[test_result][False]
          
    numerator = likelihood_of_being_right * prior_probability
    denominator = (likelihood_of_being_right * prior_probability) + (likelihood_of_being_wrong * (1 - prior_probability))
    
    posterior_probability = numerator / denominator
    
    return posterior_probability

In [13]:
def compare_profiles(profile1, profile2):
    u1 = np.array(list(profile1.values()))
    u2 = np.array(list(profile2.values()))
    limit = 7
    
    # Ref: https://en.wikipedia.org/wiki/Kullback%E2%80%93Leibler_divergence#Definition
    px = [1/np.power(2,x) for x in u1]    
    qx = [1/np.power(2,x) for x in u2]
    
    p = np.array(qx)/np.array(px)
    q = np.array(px)/np.array(qx)
    dklp = (qx * np.log2(p)).sum()
    dklq = (px * np.log2(q)).sum()
    
    t = dklp < limit and dklp >= -limit and dklq < limit and dklq >= -limit
    
    return {'test': t, 'dklp': dklp, 'dklq': dklq}

compare_profiles(user_transition_score['merchant512'], user_transition_score['merchant774'])

{'dklp': -3.5815455629843957e-32, 'dklq': 1.9965364372215852e-26, 'test': True}

In [14]:
def remove_from_classification(candidate_name, behaviour_type_table):
    cleaneds = []
    empties = []
    for be, be_list in behaviour_type_table.items():
        if candidate_name in behaviour_type_table[be]:
            behaviour_type_table[be].remove(candidate_name)
            cleaneds.append(be)
        if len(behaviour_type_table[be]) == 0:
            empties.append(be)
    for e in empties:
        del behaviour_type_table[e]
            
    return cleaneds           
            

def classify_candidates(candidate_name, behaviour_type_table, score):
    potential_matching_type = {}
    passing_score = 0.85
    sample_size = 20
    small_size_adjustment = 2
    
    for be, be_list in behaviour_type_table.items():
        be_samples = random.sample(be_list, min(len(be_list), sample_size))

        post = 0.1 # this is the prior
        for idx in range(len(be_samples)):
            y = be_samples[idx]
            result = compare_profiles(score[candidate_name], score[y])
            post = update_probability(post, flat_lookup, result['test'])
            
        if post >= passing_score * (min(small_size_adjustment,max(1,len(be_samples)))/small_size_adjustment):
            potential_matching_type[be] = post


    if len(potential_matching_type) == 0:
        
        new_class_name = max(0,len(list(behaviour_type_table.values()))) + 1
        return new_class_name
    else:
        return max(potential_matching_type, key=potential_matching_type.get)

def add_candidate_to_behaviour_type(candidate_name, matching_class, behaviour_type_table):  
    if matching_class not in behaviour_type_table.keys():
        behaviour_type_table[matching_class] = []

    if candidate_name not in behaviour_type_table[matching_class]:
        behaviour_type_table[matching_class].append(candidate_name)
        
    return candidate_name
    
def classify_users_in_list(unclassified_user_lists, behaviour_type_table, score):
    # select one user
    candidate_name = random.choice(unclassified_user_lists)
    if candidate_name:
        # classify user
        cleanup = remove_from_classification(candidate_name, behaviour_type_table)
        matching_class = classify_candidates(candidate_name, behaviour_type_table, score)

        # add the user to the proper type
        add_candidate_to_behaviour_type(candidate_name, matching_class, behaviour_type_table)
        unclassified_user_lists.remove(candidate_name)
    


In [15]:
if random_seed:
    random.seed(random_seed)

behaviour_type_table = {}
unclassified_user_lists = random.sample(list(user_transition_score.keys()), len(list(user_transition_score.keys())))

In [16]:
%%time
# while there are unclassified users
while len(unclassified_user_lists):
    classify_users_in_list(unclassified_user_lists, behaviour_type_table, user_transition_score)

for k in behaviour_type_table.keys():
    type_average = np.mean([sum(user_transition_score[x].values()) for x in behaviour_type_table[k]])
    print(k, type_average, len(behaviour_type_table[k]), cheat_lookup_all_users(behaviour_type_table[k]))

1 92.5035094643 2 {'buyer': 2}
2 66.2312140856 2 {'buyer': 2}
3 159.683051915 2 {'merchant': 2}
4 49.0083764007 2 {'merchant': 2}
5 12.281930027 2 {'attacker': 1, 'buyer': 1}
6 98.1209231372 2 {'merchant': 2}
7 102.049214135 2 {'buyer': 2}
8 67.1559866878 2 {'merchant': 2}
9 101.922277173 2 {'buyer': 2}
10 132.561958182 2 {'merchant': 2}
11 102.149457299 2 {'buyer': 2}
12 218.80413772 2 {'merchant': 2}
13 229.462406356 2 {'buyer': 2}
14 73.4408723769 2 {'merchant': 2}
15 217.471015001 2 {'buyer': 2}
16 106.275276676 2 {'merchant': 2}
17 140.871766887 2 {'buyer': 2}
18 156.883408483 2 {'merchant': 2}
19 180.067874173 2 {'merchant': 2}
20 92.4423409603 2 {'buyer': 2}
21 103.144616759 2 {'merchant': 2}
22 68.0148248979 2 {'merchant': 2}
23 226.767797246 2 {'buyer': 2}
24 93.5856756162 2 {'buyer': 2}
25 182.916292319 2 {'buyer': 2}
26 58.6583150102 2 {'merchant': 2}
27 120.816156176 2 {'buyer': 2}
28 193.815375918 2 {'merchant': 2}
29 82.5093565226 2 {'buyer': 2}
30 329.749606684 2 {'merch

In [17]:
day1_data.loc[day1_data['user'].isin(behaviour_type_table[16])]['user'].unique()

array(['merchant3159', 'merchant4030'], dtype=object)

In [25]:
a = 'merchant3159'
b = 'merchant4030'

print(user_transition_score[a])
print(user_transition_score[b])
print('compare test', compare_profiles(user_transition_score[a], user_transition_score[b]))

{'login': 5.109480743474441, 'view_item': 49.51259486024851, 'buy_item': 0, 'sell_item': 59.3730668621115, 'logout': 11.615169813843604, 'end': 12.281930026955443, 'comment': 0, 'home': 0, 'view_profile': 0, 'bank_modify': 0, 'update_address': 0, 'update_email': 0, 'password_reset': 0, 'payment_modify': 0}
{'login': 5.109480743474441, 'view_item': 16.504198286749503, 'buy_item': 0, 'sell_item': 29.14753217505511, 'logout': 11.615169813843604, 'end': 12.281930026955443, 'comment': 0, 'home': 0, 'view_profile': 0, 'bank_modify': 0, 'update_address': 0, 'update_email': 0, 'password_reset': 0, 'payment_modify': 0}
compare test {'test': True, 'dklp': 0.00035516310594054299, 'dklq': -4.1141090311792598e-14}


# Day 2, Let's see if we can find something

We are now Day 2. Surprisal values are based on a different day, but if the normal distribution is anything like Day 1, the surprisal value calculated should be still relevant.

Let's generate a new day of logs

In [19]:
start_time = datetime(2019,1,2,0,0)
number_of_new_users = 20
existing_users = user_lists[:]

if random_seed:
    random.seed(random_seed + 1)

user_list_day2s = generate_userlist(number_of_new_users, existing_users)

if random_seed:
    random.seed(random_seed + 1)
day2_logs = generate_logs(user_list_day2s, start_time)

print(len(day2_logs), 'logs events generated for', len(user_list_day2s), 'users')

day2_data = transform_logs_to_pandas(day2_logs)

105219 logs events generated for 5020 users


In [20]:
user_transition_score_day2 = get_user_transition_score(day2_data, transition_surprisal, 'user', 'path', 'success')

This is where we use our log stream analyser. This is basically a state machine that keep an ongoing total of each users. 

The theory is that if a user only do normal actions, the sum of the surprisal value of all his actions should be fairly low (under ~10... but this is arbitrary). Anything over 10 would mean that a high number of unlikely actions were performed. Lets see if we can identify which users stands out.

In [21]:
cumulative_score = [[v,sum(user_transition_score_day2[v].values())] for v in [k for k in list(user_transition_score_day2.keys())]]

df_cumulative_score = pd.DataFrame(cumulative_score, columns=['user', 'surprisal'])

avg = df_cumulative_score['surprisal'].mean()
std = df_cumulative_score['surprisal'].std()
df_cumulative_score['z'] = (df_cumulative_score['surprisal'] - avg) / std

In [22]:
df_cumulative_score.loc[df_cumulative_score['z'] >= 2].sort_values(by=['surprisal'], ascending=False)

,user,surprisal,z
2508,bot2142,38553.406663,35.086411
2473,bot900,37788.333968,34.387060
1558,bot1946,34006.378875,30.929984
3520,bot2306,21911.719044,19.874284
3615,bot2203,19746.726100,17.895269
3452,bot776,19621.927008,17.781191
2875,bot2816,16806.134478,15.207282
1660,bot672,13864.890633,12.518698
1720,bot2886,11748.732116,10.584322
1334,bot3972,7516.415083,6.715571


In [23]:
if random_seed:
    random.seed(random_seed)

unclassified_user_lists = random.sample(list(user_transition_score_day2.keys()), len(list(user_transition_score_day2.keys())))

In [24]:
%%time
# while there are unclassified users
while len(unclassified_user_lists):
    classify_users_in_list(unclassified_user_lists, behaviour_type_table, user_transition_score_day2)

for k in behaviour_type_table.keys():
    type_average = np.mean([sum(user_transition_score[x].values()) for x in behaviour_type_table[k]])
    print(k, type_average, len(behaviour_type_table[k]), cheat_lookup_all_users(behaviour_type_table[k]))

2 74.2129720117 2 {'buyer': 2}
3 258.170982493 1 {'merchant': 1}
5 35.8402315907 2 {'buyer': 2}
6 98.1209231372 2 {'merchant': 2}
10 44.3301214569 1 {'merchant': 1}
12 55.9452912707 1 {'merchant': 1}
16 147.248752194 2 {'merchant': 2}
18 156.883408483 2 {'merchant': 2}
19 180.067874173 2 {'merchant': 2}
21 82.1209271013 1 {'merchant': 1}
24 243.466132609 1 {'buyer': 1}
25 182.916292319 2 {'buyer': 2}
27 96.0744752202 2 {'buyer': 2}
28 252.936461723 1 {'merchant': 1}
29 78.7141666312 2 {'buyer': 2}
30 342.231064734 1 {'merchant': 1}
31 97.6665593238 2 {'merchant': 2}
34 276.764906678 2 {'buyer': 2}
35 119.733990024 2 {'buyer': 2}
38 111.231678021 2 {'merchant': 2}
40 119.849590428 2 {'buyer': 2}
42 137.764106976 2 {'buyer': 2}
48 82.1209271013 1 {'merchant': 1}
52 84.0148209338 1 {'merchant': 1}
53 75.6704663017 2 {'spammer': 2}
56 77.3075331791 2 {'buyer': 2}
64 217.404316004 1 {'merchant': 1}
65 54.6160442717 2 {'buyer': 2}
66 89.1736309969 2 {'buyer': 2}
67 179.868960564 1 {'buyer': 

KeyError: 'merchant5001'

## Why these users are identified as outliers?

If we look at the Top 1 outlier, we can see each action performed, and the surprisal value of each action.

If we look at a normal user, we can see that the surprisal value assigned to each value is really low, only slightly contributing to give that user a high score.

In [ ]:
df_cumulative_score.sort_values(by=['surprisal'], ascending=False).tail()

# Day 3: Automatically Classifying Users

At one point, we still need to classify behaviours. Using Naive Bayes over the actions, we can score each users to the likely category they belong too.

Could we have done that without looking for the outliers? Yes, but we still need to identify which users is normal and which one is likely not.

At first, we need to calculate the probability distribution of each actions for each categories.

We can now use this probability distribution over a log stream